In [1]:
import matplotlib.pyplot as plt
import numpy as np
from netCDF4 import Dataset
import matplotlib as mpl
import matplotlib.tri as tri
import datetime 
import time
#import netcdftime
import sys
import glob
import pandas as pd
from matplotlib.colors import LinearSegmentedColormap
import os
from scipy.interpolate import griddata
import csv
#from mpl_toolkits.basemap import Basemap
from scipy import interpolate
import xarray as xr

In [2]:
colors_orig = [ (103,0,31),(178,24,43),(214,96,77),(244,165,130),(253,219,199),(255,255,255),(209,229,240),(146,197,222),(67,147,195),(33,102,172),(5,48,97)]
colorsm=[[a/255. for a in x] for x in colors_orig]
div_color= LinearSegmentedColormap.from_list( 'div_color', colorsm, N=20)
level=np.arange(-1,1.1,.1) #-.6,.5,.05)  #
bounds=level.tolist()
cmap = div_color
cmaplist = [cmap(i) for i in range(cmap.N,0,-1)]
cmap = mpl.colors.LinearSegmentedColormap.from_list('Custom cmap', cmaplist, cmap.N)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

In [7]:
datafile = xr.open_dataset("../Masterarbeit/crosssection_data/fc_DOM01_0002_URB_temp.nc")
#datafile = datafile.isel(height=slice(0, 60))
#a = datafile.variables['T'][0,:,:]

FileNotFoundError: [Errno 2] No such file or directory: b'/data/hubhome/s6mabohm/Masterarbeit/Masterarbeit/crosssection_data/fc_DOM01_0002_URB_temp_grid.nc'

In [416]:
#flipped_datafile = datafile.reindex(height=datafile['height'][::-1])
#datafile = flipped_datafile.isel(height=slice(0, 60))
#t = datafile.variables['T'][0,:,:]

In [7]:
res='c'
lonc=8.
latc=51.

cor = xr.open_dataset('../Masterarbeit/crosssection_data/extpar_icon_grid_0026_R03B07_G.nc')
luf = cor.variables['LU_CLASS_FRACTION'][:,:]
topo = cor.variables['topography_c'][:]
#map_lim_llon=0.
#map_lim_llat=44.
#map_lim_ulon=16.
#map_lim_ulat=55.
ind=1

FileNotFoundError: [Errno 2] No such file or directory: b'/data/hubhome/s6mabohm/Masterarbeit/Masterarbeit/crosssection_data/extpar_icon_grid_0026_R03B07_G.nc'

In [22]:
#centers
#gridfile = xr.open_dataset("./gridfile.nc")
clons  = np.rad2deg(datafile.variables["clon"])[::-1] 
clats  = np.rad2deg(datafile.variables["clat"])[::-1] 

NameError: name 'datafile' is not defined

In [8]:
hhl_file = xr.open_dataset("../Masterarbeit/crosssection_data/hhl_grid.nc")
hhl = hhl_file.variables["HHL"][0,:,:][::-1]
hhl_f = hhl
### Full levels: because the levels are defined at half levels, it is corrected here
for k in range(1,hhl.shape[0]):
    hhl_f[k,:] = hhl[k,:] + (hhl[k,:]-hhl[k-1,:])/2


FileNotFoundError: [Errno 2] No such file or directory: b'/data/hubhome/s6mabohm/Masterarbeit/Masterarbeit/crosssection_data/hhl_grid.nc'

In [479]:
lar1 = 51
lar2 = 53
lor1 = 13
lor2 = 14

ind=[]
for i in range(0,clons.shape[0]):
    if ((clats[i]<lar2 and clats[i]>lar1) and (clons[i]>lor1 and clons[i]<lor2)): #find the indexes and select the values after
        ind.append(i)

In [480]:
lo_i = 13
la_i = 51
lo_f = 14
la_f = 53

lo_line = np.linspace(lo_i,lo_f,60)
la_line = np.linspace(la_i,la_f,60)
cross_line = np.vstack(( np.linspace(lo_i,lo_f,60), np.linspace(la_i,la_f,60) )).T
interp_method = 'cubic'
c = []
### Cross line as tuples
for i in range(1,cross_line.shape[0]+1):
    c.append(i)
lons=clons[ind]
lats=clats[ind]
lons=lons
i_c=np.vstack((lons,lats)).T

In [481]:
cross_val=np.zeros((hhl_f.shape[0]-1,len(cross_line)))
cross_fl =np.zeros((hhl_f.shape[0]-1,len(cross_line)))
cross_test =np.zeros((hhl_f.shape[0]-1,len(cross_line)))

In [482]:
# HORIZONTAL INTERPOLATION
for i in range(t.shape[0]-1,0,-1):
    hhlf_i = griddata(i_c,hhl_f[i,ind],cross_line,method=interp_method)
#    print(i, hhlf_i)
    test = griddata(i_c,t[i,ind],cross_line,method=interp_method)
#    print(i,test)
    val = test - np.nanmean(test)
#    print(val)
    cross_val[i,:] = val
    cross_fl[i,:] = hhlf_i
    cross_test[i,:] = c

In [1]:
# VERTICAL INTERPOLATION
#z = np.linespace
x_labels = []
cross_line_lab = np.vstack([ np.linspace(lo_i,lo_f,60), np.linspace(la_i,la_f,60) ]).T
#for pair in cross_line:
    #'{:.2f}'.format(pair[0])
x_labels = [['{:.2f}'.format(pair[0]),'{:.2f}'.format(pair[1])] for pair in cross_line_lab]
print(x_labels)

NameError: name 'np' is not defined

In [2]:
fig, ax =plt.subplots( figsize=(10, 10))
plt.contourf(cross_test,cross_fl,cross_val,20,cmap=cmap)

## interp of invar fields:
topo_i = griddata(i_c,topo[ind],cross_line,method=interp_method)
lu_i = griddata(i_c,luf[18,ind],cross_line,method=interp_method)

plt.fill_between(c,-5,topo_i,color='k')

lu_urb = lu_i
for i in range(0,lu_i.shape[0]):
  if (lu_urb[i]<.7):
      lu_urb[i]= np.nan
  else:
      lu_urb[i]=20
plt.fill_between(c,-5,lu_urb,color='r')
#plt.ylim(15068.11000195,52630.90625)
plt.ylim(-10,15000)
#plt.xlim(30,30)
plt.xticks(c[5::5],x_labels[5::5],rotation=45)
plt.colorbar()
plt.show()

NameError: name 'plt' is not defined